In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from datetime import time, datetime
import pandas as pd

In [ ]:
# !which chromedriver

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path)
browser = Browser('chrome', **executable_path, headless=False)
# browser = Browser('chrome')

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147


[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/84.0.4147.30/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\a36s7zz\.wdm\drivers\chromedriver\win32\84.0.4147.30]


In [3]:
base_url = 'https://www.walmart.com'
search_url = '/search/?query=room%20air%20purifier'
browser.visit(base_url + search_url)

In [4]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [94]:
# trying to find the 'Free pickup' element and click it
# found it but can not click
# trying to find what parts of html changes during click
# didn't find any changes during click
# moving on from this issue

pickup_delivery = soup.find('div', class_='radio-button-facet').find_all('div')[0].find('input')
# pickup_delivery['value'] = 'checked'
# pickup_delivery = soup.find('div', class_='radio-button-facet').find_all('label')[3].find('input', class_='input-toggle__input')
print(pickup_delivery)

<input checked="" class="input-toggle__input" type="radio" value=""/>


In [93]:
# asking user to see if they want to only see 'Free pickup' deivices (aka in store)
# moving on from this issue due to previous cell

instore = input("Do you want to only scrape product found in store? [y/n]: ")
if (instore != 'y') and (instore != 'n'):
    print("Please only reply with only a 'y' or a 'n'")
    instore = input("Do you want to only scrape product found in store? [y/n]: ")
    
if instore == 'y':
    pickup_delivery.click()
    print(pickup_delivery)

Do you want to only scrape product found in store? [y/n]: y


AttributeError: 'NoneType' object has no attribute 'click'

In [48]:
# Create empty lists for scraped data to be stored in.
productURL_list = []
productImage_list = []
productTitle_list = []
starReview_list = []
reviewAmount_list = []
currentPrice_list = []
reviewURL_list = []
freepickup_list = []

# loop through all of the pages in this search
listOfPages = soup.find('ul', class_='paginator-list').find_all('li')
numberOfPages = listOfPages[0].find('a')['aria-label'].split()[3]
print(f"the number of pages to loop through is {numberOfPages}")

# datetime object containing current date and time
now = datetime.now()

for i in range(1,int(numberOfPages)+1):
# for i in range(1,2):
    
    # click on the next page number and scrape the html
    page_link = browser.links.find_by_text(i).click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    # loop through all of the products on that page
    products = soup.find_all('div', class_='search-result-gridview-item')
    print(f"quering page {i}...............")

    for product in products:

        # Get the link to the product page.
        link = product.find('a')
        href = link['href']
        product_url = base_url + href
        productURL_list.append(product_url)

        # Get the link to the product image.
        img = product.find('img')['src']
        productImage_list.append(img)

        # Get the product title.
        product_title = product.find('img')['alt']
        productTitle_list.append(product_title)

        # Try to get the number of reviews, there may not be any.
        try:
            review_amount = product.find('span', class_='seo-review-count visuallyhidden').text
            reviewAmount_list.append(review_amount)

        except:
            review_amount = 0
            reviewAmount_list.append(review_amount)

        # In the case where there are at least one review...
        if int(review_amount) > 0:
            # Get the average number of stars
            stars_review = product.find('span', class_='visuallyhidden seo-avg-rating').text   
            starReview_list.append(stars_review)

            # Get the URL to the reviews section for that product.
            review_url = product.find('div', class_='stars').find('a')['href']
            reviewURL_list.append(base_url+review_url)

        # Otherwise, use defalt 0 or NaN values for these entries.
        else:
            stars_review = 0
            starReview_list.append(stars_review)

            review_url = "NaN"
            reviewURL_list.append(review_url)

        # Get the price of the product.
        try:
            price = product.find('span', class_='price-main-block')
            current_price = price.find('span', class_='visuallyhidden').text
        # Some products don't have a specific price, or price is only shown in cart
        except:
            price = product.find('span', class_='search-result-productprice')
            current_price = price.find('span', class_='visuallyhidden').text
        
        currentPrice_list.append(current_price)
        
        # Check to see if the product has 'Free pickup' AKA in store
        # Look at the div under the price div
        try:
            shipping_details = product.find('div', class_='search-result-product-shipping-details')
            delivery_options_list = []
            # loop through all possile delivery options displayed
            for option in shipping_details.children:
                option.span.unwrap() #unwrap takes the span tag off
                delivery_option = option.text
                if delivery_option == 'Free pickup': #assuming 'free pickup' means avaliable in store
                    free_pickup = True
                    delivery_options_list.append(free_pickup)
                else:
                    free_pickup = False
                    delivery_options_list.append(free_pickup)

            instore = any(delivery_options_list)
        # if there are no delivery options, assume not avaialble in store
        except:
            instore = False
        freepickup_list.append(instore)

    # Close the browser window
    # browser.quit()

    # Create a dictionary with the lists of the scrapped data.
    data = {
        "Title": productTitle_list,
        "URL": productURL_list,
        "Image": productImage_list,
        "AverageStars": starReview_list,
        "NumberofReviews": reviewAmount_list,
        "ReviewsURL": reviewURL_list,
        "Price": currentPrice_list,
        "Free Pickup": freepickup_list
           }

    # Create a Pandas DataFrame with that dictionary
    product_df = pd.DataFrame.from_dict(data)
#     product_df.head()

the number of pages to loop through is 11
quering page 1...............
<div class="free-shipping"><span class="free-label">Free</span> delivery</div>
<div class="free-pickup"><span class="free-label">Free</span> pickup</div>
[True]
<div class="marketplace-sold-by">Sold &amp; shipped by <span class="marketplace-sold-by-company-name">Keenstone Corp.</span></div>
<div class="free-shipping"><span class="free-label">Free</span> delivery</div>
[True, False]
<div class="free-shipping"><span class="free-label">Free</span> delivery</div>
<div class="free-pickup"><span class="free-label">Free</span> pickup</div>
[True, False, True]
<div class="free-shipping"><span class="free-label">Free</span> delivery</div>
<div class="free-pickup"><span class="free-label">Free</span> pickup</div>
[True, False, True, True]
<div class="free-shipping"><span class="free-label">Free</span> delivery</div>
[True, False, True, True, False]
<div class="free-shipping"><span class="free-label">Free</span> delivery</div

<div class="marketplace-sold-by">Sold &amp; shipped by <span class="marketplace-sold-by-company-name">WALFRONT LLC</span></div>
<div class="free-shipping"><span class="free-label">Free</span> delivery</div>
[True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False]
<div class="marketplace-sold-by">Sold &amp; shipped by <span class="marketplace-sold-by-company-name">SainStore Inc.</span></div>
<div class="free-shipping"><span class="free-label">Free</span> delivery</div>
[True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False, False]
<div class="marketplace-sold-by">Sold &amp; shipped by <span class="marketplace-sold-by-c

quering page 3...............
<div class="marketplace-sold-by">Sold &amp; shipped by <span class="marketplace-sold-by-company-name">Keenstone Corp.</span></div>
<div class="free-shipping"><span class="free-label">Free</span> delivery</div>
[True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False, False, False, True, True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False, False, False, True, False]
<div class="free-shipping"><span class="free-label">Free</span> delivery</div>
<div class="free-pickup"><span class="free-label">Free</span> pickup</div>
[True, False, True, True, False, True, False, True, False, False, False,

quering page 4...............
<div class="free-shipping"><span class="free-label">Free</span> delivery</div>
<div class="free-pickup"><span class="free-label">Free</span> pickup</div>
[True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False, False, False, True, True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False, False, False, True, False, True, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True]
<div class="free-shi

<div class="free-shipping"><span class="free-label">Free</span> delivery</div>
<div class="free-pickup"><span class="free-label">Free</span> pickup</div>
[True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False, False, False, True, True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False, False, False, True, False, True, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, True, True, Fals

...

quering page 7...............
<div class="marketplace-sold-by">Sold &amp; shipped by <span class="marketplace-sold-by-company-name">Pinkiou Store</span></div>
<div class="free-shipping"><span class="free-label">Free</span> delivery</div>
[True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False, False, False, True, True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False, False, False, True, False, True, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, 

quering page 8...............
<div class="marketplace-sold-by">Sold &amp; shipped by <span class="marketplace-sold-by-company-name">KWANSHOP</span></div>
<div class="free-shipping"><span class="free-label">Free</span> delivery</div>
[True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False, False, False, True, True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False, False, False, True, False, True, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False

quering page 9...............
<div class="marketplace-sold-by">Sold &amp; shipped by <span class="marketplace-sold-by-company-name">KWANSHOP</span></div>
<div class="free-shipping"><span class="free-label">Free</span> delivery</div>
[True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False, False, False, True, True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False, False, False, True, False, True, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False

quering page 10...............
<div class="marketplace-sold-by">Sold &amp; shipped by <span class="marketplace-sold-by-company-name">iSource</span></div>
<div class="free-shipping"><span class="free-label">Free</span> delivery</div>
[True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False, False, False, True, True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False, False, False, True, False, True, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False

quering page 11...............
<div class="marketplace-sold-by">Sold &amp; shipped by <span class="marketplace-sold-by-company-name">TAWC Sales LLC</span></div>
<div class="free-shipping"><span class="free-label">Free</span> delivery</div>
[True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False, False, False, True, True, False, True, True, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, True, True, False, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False, False, False, True, False, True, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False

In [49]:
product_df = product_df.astype({
    "NumberofReviews": 'int',
    "AverageStars": 'float'
})

product_df.dtypes

Title               object
URL                 object
Image               object
AverageStars       float64
NumberofReviews      int32
ReviewsURL          object
Price               object
Free Pickup           bool
dtype: object

In [68]:
ordered_df = product_df.sort_values(by=['Free Pickup', 'NumberofReviews'], ascending=False)
ordered_df.head()
# shortlist_df = product_df.sort_values(by='NumberofReviews', ascending=False)[product_df['Free Pickup']==True]
# shortlist_df.head()

,Title,URL,Image,AverageStars,NumberofReviews,ReviewsURL,Price,Free Pickup
19,Envion Ionic Pro CA 200 Compact Air Purifier M...,https://www.walmart.com/ip/Envion-Ionic-Pro-CA...,https://i5.walmartimages.com/asr/0715b6c6-03e5...,4.0,380,https://www.walmart.com/ip/Envion-Ionic-Pro-CA...,$79.94,True
59,Envion Ionic Pro CA 200 Compact Air Purifier M...,https://www.walmart.com/ip/Envion-Ionic-Pro-CA...,https://i5.walmartimages.com/asr/0715b6c6-03e5...,4.0,380,https://www.walmart.com/ip/Envion-Ionic-Pro-CA...,$79.94,True
52,"Honeywell HEPA Allergen Remover Air Purifier, ...",https://www.walmart.com/ip/Honeywell-HEPA-Alle...,https://i5.walmartimages.com/asr/3574cc6b-93c9...,4.6,261,https://www.walmart.com/ip/Honeywell-HEPA-Alle...,$220.99,True
12,"Honeywell HEPA Allergen Remover Air Purifier, ...",https://www.walmart.com/ip/Honeywell-HEPA-Alle...,https://i5.walmartimages.com/asr/3574cc6b-93c9...,4.6,261,https://www.walmart.com/ip/Honeywell-HEPA-Alle...,$220.99,True
42,"Filtrete by 3M Room Air Purifier, Console, 110...",https://www.walmart.com/ip/Filtrete-by-3M-Room...,https://i5.walmartimages.com/asr/eae28c2f-5dbe...,4.6,228,https://www.walmart.com/ip/Filtrete-by-3M-Room...,$49.94,True


In [69]:
# product_df.describe()
# product_df.head(6)
ordered_df.describe()

,AverageStars,NumberofReviews
count,435.000000,435.000000
mean,2.176940,31.082759
std,2.227277,82.806852
min,0.000000,0.000000
25%,0.000000,0.000000
50%,1.000000,1.000000
75%,4.550000,11.000000
max,5.000000,548.000000


In [71]:
# dd/mm/YY H:M:S
dt_string = now.strftime("%d_%m_%Y__%H_%M_%S")
ordered_df.to_csv(f"WalmartRAPScrape_{dt_string}.csv")

In [ ]:
print(product_df['URL'][0])
print('----------------------------------------------------------------------------------------------------')
print(product_df['ReviewsURL'][5])

In [ ]:
# LEFT OFF HERE
# WANT TO CLICK BY TEXT LIKE IN ABOVE QUERY, DIDN'T WORK THOUGH...
# THIS WILL LET ME GO TO EACH PAGE OF THE REVIEWS, 
# AND I THINK IT WILL LET ME LOAD ALL OF THAT PAGES REVIEWS

# Get reviews 

review_df = pd.DataFrame()

# loop through all of the products in the product dataframe
# for i in range(len(product_df["ReviewsURL"])):
for i in range(4,6):
    if product_df['NumberofReviews'][i] > 0:
        
        productTitleList = []
        reviewURLList = []
        reviewTitleList = []
        reviewRatingList = []
        reviewCommentList = []
    
#         try:
        browser.visit(product_df["ReviewsURL"][i])
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        allReviewsURL = soup.find('a', class_="button ReviewBtn-container ReviewsHeader-seeAll button--primary")['href']
        browser.visit(base_url+allReviewsURL)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        
        # loop through all of the pages in this search
        reviewPages = soup.find('ul', class_='paginator-list').find_all('li')
        pagesOfReviews = reviewPages[-1].find('button').text
        print(f"the number of pages to loop through is {pagesOfReviews}")
        for j in range(1, int(pagesOfReviews)+1):
            
            page_link = browser.links.find_by_text(j).click()
            html = browser.html
            soup = BeautifulSoup(html, 'html.parser')
            reviews = soup.find_all('div', class_='ReviewList-content')

            for review in reviews:

                productTitleList.append(product_df['Title'][i])
                reviewURLList.append(product_df['ReviewsURL'][i])

                try:
                    title = review.find('h3', class_='review-title').text
                    reviewTitleList.append(title)
                    print(title)

                except:
                    title = 'None'
                    reviewTitleList.append(title)
                    print(title)
#                         pass

#                 # adding try only because the browser needs to scroll
#                 # once this is added, there wont need to be a try
#                 # each review has to have a star value
#                 try:
#                     reviewRating = review.find('span', class_='seo-avg-rating').text
#                     reviewRatingList.append(reviewRating)
#                     print(reviewRating)
#                 except:
#                     pass

#                 # adding try and except for the review body for same reason as stars
#                 try:
#                     reviewComment = review.find('p').text
#                     reviewCommentList.append(reviewComment)
# #                         print(reviewComment)
#                 except:
#                     pass


#             # only until I fix the scroll
#             reviewTitleList.pop()
#             productTitleList.pop()
#             reviewURLList.pop()

#             data = {
#                 'ProductTitle': productTitleList,
#                 'ReviewURL': reviewURLList,
#                 'ReviewTitle': reviewTitleList,
#                 'ReviewStarRating': reviewRatingList,
#                 'ReviewComment': reviewCommentList
#             }
#             print('stored data')
#             productReviews_df = pd.DataFrame.from_dict(data)
#             productReviews_df.head()
#             print('into dataframe')

#         except:
#             pass
    
#         print('add to existing dataframe')
#         review_df = review_df.append(productReviews_df, ignore_index=True)
# print('done')
# #                 browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [ ]:
# len(review_df)
# review_df.tail()
review_df.describe()

In [ ]:
reviews = soup.find('div', class_='review-highlight')
positive = reviews.find('div', class_='font-bold highlight-title').text
stars = reviews.find('span', class_='seo-avg-rating').text
# starts = reviews.find('span', class_='seo-average-rating')
body = reviews.find('div', class_='collapsable-content-container').text
print(positive)
print(stars)
print(body)

In [ ]:
reviews = soup.find_all('div', class_='ReviewList-content')

In [ ]:
# Get reviews 

review_df = pd.DataFrame()

# for i in range(len(product_df["ReviewsURL"])):
for i in range(4,5):
    if product_df['NumberofReviews'][i] > 0:
#     number_of_reviews = product_df['NumberofReviews'][i]
#     for j in range(number_of_reviews):
        
        productTitleList = []
        reviewURLList = []
        reviewSiteLink = []
        reviewTitleList = []
        reviewRatingList = []
        reviewCommentList = []
        
#         if len(reviewRatingList) < product_df['NumberofReviews'][i]:
        
#         while len(reviewRatingList) < product_df['NumberofReviews'][i]:
    
        try:
            # go to the product url link
            browser.visit(product_df["ReviewsURL"][i])
            html = browser.html
            soup = BeautifulSoup(html, 'html.parser')
            
            # find the 'see all reviews' button and go there
            allReviewsURL = soup.find('a', class_="button ReviewBtn-container ReviewsHeader-seeAll button--primary")['href']
            browser.visit(base_url+allReviewsURL)
            soup = BeautifulSoup(html, 'html.parser')
            
            # find the list of reviews
            reviews = soup.find_all('div', class_='ReviewList-content')
            
#             for review in reviews:
            for j in range(len(reviews)):
                reviewSiteLink.append(review)
                reviewSiteLink[j].send_keys(Keys.PAGE_DOWN)
                soup = BeautifulSoup(html, 'html.parser')
                reviews = soup.find('div', class_='ReviewList-content')
                
#                 print(f"j={j}")
                productTitleList.append(product_df['Title'][i])
                reviewURLList.append(product_df['ReviewsURL'][i])
                title, stars, comments = scrapeReviews(review)
                reviewTitleList.append(title)
                reviewRatingList.append(stars)
                reviewCommentList.append(comments)


                data = {
                    'ProductTitle': productTitleList,
                    'ReviewURL': reviewURLList,
                    'ReviewTitle': reviewTitleList,
                    'ReviewStarRating': reviewRatingList,
                    'ReviewComment': reviewCommentList
                }
                print('stored data')
                productReviews_df = pd.DataFrame.from_dict(data)
                productReviews_df.head()
                print('into dataframe')

        except:
            pass

        print('add to existing dataframe')
        review_df = review_df.append(productReviews_df, ignore_index=True)
print('done')

In [ ]:
# len(review_df)
# review_df.tail()
review_df.describe()

In [ ]:
def scrapeReviews(review):
    
    # get the title of the review
    try:
        title = review.find('h3', class_='review-title').text
    except:
        title = 'None'
        
    # get the number of stars in this review
    reviewRating = review.find('span', class_='seo-avg-rating').text
    
    # get the context of the review
    reviewComment = review.find('p').text
        
    # return these items as a tuple
    return title, stars, comments

In [ ]:
browser.quit()

In [ ]:
soup.execute_script("window.scrollTo(0, document.body.scrollHeight);")